In [449]:
import numpy as np
from functools import reduce
import torch

In [434]:
def permutations(arr):
    if not arr : return [[]]
    res = []
    for i in range(len(arr)):
        curr = arr[i]
        others = arr[:i] + arr[i+1:]
        for permutation in permutations(others):
            res.append([curr] + permutation)
    return res

def add(a, b) : return a + b
def mult(a, b) : return a * b

def int_to_bitarray(input, n) : return [int(bit) for bit in bin(input)[2:].zfill(n)]

In [604]:
indices = [
  [0,1],
  [2,3],
  [4,5],
  [1,2],
  [3,4],
  [0,1],
  [2,3],
  [4,5],
  [1,2],
  [3,4]
]
n = 3
m = 3

At = torch.zeros((8,8), dtype=torch.complex128)
for i in range(7) : At[i, i] = 1
At[7,7] = -1

In [547]:
indices = [
  [0,2],
  [1,3],
  [0,1],
  [2,3]
]
n = 2
m = 2

At = torch.tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]], dtype=torch.complex128)

In [605]:
# HELPER FUNCTIONS

def hadamard(param, i, j, N) :
  res = torch.zeros((N,N), dtype=torch.complex128)
  for k in range(N) :
    for l in range(N) :
      if k == i and l == i : res[k,l] = torch.cos(param / 2)
      elif k == i and l == j : res[k,l] = 1j * torch.sin(param / 2)
      elif k == j and l == i : res[k,l] = 1j * torch.sin(param / 2)
      elif k == j and l == j : res[k,l] = torch.cos(param / 2)
      elif k == l : res[k,l] = 1
  return res

def get_U(phases, thetas, n, m, indices) : 
  U0 = torch.zeros((n+m, n+m), dtype=torch.complex128)
  for i in range(n+m) : 
    if i < n : 
      U0[i, i] = torch.exp(1j * phases[i])
    else :
      U0[i, i] = 1
  matrices = [hadamard(param, i[0], i[1], n + m) for param, i in zip(thetas, indices)]#[::-1]
  U = reduce(lambda x, y : torch.matmul(y, x), matrices, U0)
  return U

def get_A(U, n, m) : 
  A = torch.zeros((2**n,2**n), dtype=torch.complex128)
  for row in range(2**n) :
    row_bits = int_to_bitarray(row, n)
    for col in range(2**n) : 
      col_bits = int_to_bitarray(col, n)
      if sum(row_bits) == sum(col_bits) :
        in_indices = [i for i, val in enumerate(col_bits) if val] + list(range(n, n+m))
        out_indices = [o for o, val in enumerate(row_bits) if val] + list(range(n, n+m))
        #print("row: ", row, "col: ", col, [[[i,j] for i, j in zip(in_indices, oi)] for oi in permutations(out_indices)])
        A[row, col] = reduce(torch.add, [reduce(torch.multiply, [U[i, j] 
                        for i, j in zip(in_indices, oi)]) for oi in permutations(out_indices)])
  return A

def get_fidelity(A, At) : 
  B = A / A[0,0]
  numer = torch.trace(B.conj() @ At) * torch.trace(At.conj() @ B) 
  denom = torch.trace(B.conj() @ B) * torch.trace(At.conj() @ At)
  fidelity = numer / denom
  return fidelity.real

def get_dist(A, At) :
  B = A / A[0, 0]
  return torch.sum(torch.square(torch.abs(B - At)))

def get_success(A) : return A[0, 0].real ** 2

In [611]:
g = torch.Generator().manual_seed(3141592653)
phases = 2 * torch.pi * torch.rand(n, generator=g, dtype=torch.float64) - torch.pi
thetas = 2 * torch.pi * torch.rand(len(indices), generator=g, dtype=torch.float64) - torch.pi

for param in [phases, thetas] : param.requires_grad = True

#phases = torch.tensor([torch.pi, torch.pi])
#thetas = torch.tensor([2*54.74 * torch.pi / 180, 2*54.74 * torch.pi / 180, 2*-54.74 * torch.pi / 180, 2*17.63 * torch.pi / 180], requires_grad=True, dtype=torch.float64)
optimizer = torch.optim.SGD([phases, thetas], 0.05)

In [617]:
optimizer = torch.optim.SGD([phases, thetas], 0.01)

In [618]:
for step in range(500) :

  U = get_U(phases, thetas, n, m, indices)
  A = get_A(U, n, m)

  fidelity = get_fidelity(A, At)
  success = get_success(A)
  distance = get_dist(A, At)
  
  #loss =  get_dist(A, At)
  loss = (fidelity - 1) ** 2 + 1e-5 * (success - 1) ** 2
  #print(step, loss.item())
  if step % 50 == 0 : print(step, fidelity.item(), success.item(), distance.item())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

print("Optimized phases:", phases.detach().numpy() % (2 * np.pi) - np.pi)
print("Optimized thetas:", thetas.detach().numpy() % (2 * np.pi) - np.pi)
print("Fidelity:", fidelity.item())
print("Success:", success.item())

0 0.8459407270597069
50 0.9801555529941426
100 0.999751669761665
150 0.9999974205137899
200 0.9999999212483678
250 0.9999999466327658
300 0.999999946890236
350 0.9999999468932375
400 0.99999994689317
450 0.9999999468930317
Optimized phases: [ 3.14159265e+00 -3.55271368e-15  3.55271368e-15]
Optimized thetas: [-2.11327576 -0.38009807  1.26824939  0.98771758 -0.16621758  2.10052575
  0.77861283  3.05156843 -2.49794873 -0.61153585]
Fidelity: 0.9999999468929395
Success: 0.0038549096438747913


In [619]:
get_fidelity(A, At)

tensor(1.0000, dtype=torch.float64, grad_fn=<SelectBackward0>)

Optimized phases: [ 3.14159265e+00 -3.55271368e-15  3.55271368e-15]
Optimized thetas: [-2.11327576 -0.38009807  1.26824939  0.98771758 -0.16621758  2.10052575
  0.77861283  3.05156843 -2.49794873 -0.61153585]
Fidelity: 0.9999999468929395
Success: 0.0038549096438747913